In [1]:
# cleaning texts using regular expression
import re
def preprocessor(text):
    text = re.sub('<[^>]*>','',text)             # remove <...> (tags)
    text = re.sub('[\W_]+',' ',text)             # remove all non-words
    text = text.lower()                # change to lower cases
    return text

In [2]:
import pandas as pd
import numpy as np

# read dataset "kor_movie.csv"
df_kor = pd.read_csv("newss.csv", encoding='utf-8')

df_kor

,Unnamed: 0,날짜,언론사,제목,sentiment
0,0,2022.01.14,한국경제,"머스크 ""도지코인, 테슬라 결제 시작""…순식간에 20% 폭등",1.0
1,1,2022.01.14,조선비즈,"테슬라, 전기 픽업트럭 생산 또 연기…“2022년 생산” 홈피 문구 삭제",0.0
2,3,2022.01.14,이데일리,10대 독일 청년 “13개국서 테슬라 차량 25대 원격 해킹”,0.0
3,4,2022.01.14,디지털데일리,10대 해커에 뚫린 테슬라…원격시동에 차량문 개폐,0.0
4,6,2022.01.14,한국경제TV,금리인상 우려·빅테크 급락에 美증시 하락…테슬라 6%↓ [출근전 꼭 글로벌브리핑],0.0
...,...,...,...,...,...
1671,16731,2022.01.15,뉴스1,테슬라 전기트럭 출시 연기에도 1.75% 상승 마감,1.0
1672,16732,2022.01.15,글로벌경제,"테슬라 애플 엔비디아 등 성장주 일제히 상승...뉴욕증시, 보합세 마감",1.0
1673,16733,2022.01.15,테크M,"[글로벌] 전세계 130만명 예약했는데...테슬라, 사이버트럭 '또' 연기",0.0
1674,16736,2022.01.15,초이스경제,"미국증시 자동차주...테슬라 · 루시드 '상승', 리비안 '하락'",1.0


In [3]:
from sklearn.model_selection import train_test_split

filtered_df = df_kor.dropna()
df_sample = filtered_df.iloc[:]

X = df_sample['제목'] 
y = df_sample['sentiment']

# Split Dataset into 70% train and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_train.shape

(1172,)

In [4]:
# function for tokenizing + stemming using Okt
from konlpy.tag import Okt

okt = Okt()

# tokenizer_kor function perform tokenizing + stemming using Okt
def tokenizer_kor(text):
    return okt.morphs(text, norm=True, stem=True)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorize X_train - TF-IDF Matrix (with preprocessing, tokenizing, max_df = 0.1)
tfidf = TfidfVectorizer(preprocessor=preprocessor, tokenizer=tokenizer_kor, max_df=0.1)
X_train_vector = tfidf.fit_transform(X_train)
X_test_vector = tfidf.transform(X_test)

/Users/kangjiyoung/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
# TF-IDF Matrix 
X_train_vector = X_train_vector.toarray()
X_test_vector = X_test_vector.toarray()

In [7]:
# check the review 0 vector
np.set_printoptions(threshold=np.inf)

In [8]:
from sklearn.linear_model import LogisticRegression

# LogisticRegression(penalty="l2")
lr = LogisticRegression(penalty="l2")
lr.fit(X_train_vector,y_train)

LogisticRegression()

In [9]:
# train and test accuracy of the model. use .score(X, y)
print(lr.score(X_train_vector,y_train))
print(lr.score(X_test_vector,y_test))

0.9658703071672355
0.8747514910536779


NameError: name 'df' is not defined

In [11]:
df2 = df_sample.loc[df_sample['날짜'] == '2022.02.01']
df3 = df_sample.loc[df_sample['제목'].str.contains('테슬라')]
df3.head()
# df3['제목']

,Unnamed: 0,날짜,언론사,제목,sentiment
0,0,2022.01.14,한국경제,"머스크 ""도지코인, 테슬라 결제 시작""…순식간에 20% 폭등",1.0
1,1,2022.01.14,조선비즈,"테슬라, 전기 픽업트럭 생산 또 연기…“2022년 생산” 홈피 문구 삭제",0.0
2,3,2022.01.14,이데일리,10대 독일 청년 “13개국서 테슬라 차량 25대 원격 해킹”,0.0
3,4,2022.01.14,디지털데일리,10대 해커에 뚫린 테슬라…원격시동에 차량문 개폐,0.0
4,6,2022.01.14,한국경제TV,금리인상 우려·빅테크 급락에 美증시 하락…테슬라 6%↓ [출근전 꼭 글로벌브리핑],0.0


In [12]:
test = df3['제목'].values

# transform the test text to TF-IDF
test_tfidf = tfidf.transform(test).toarray()

# get the prediction and probability
y_pred = lr.predict(test_tfidf)
y_prob = lr.predict_proba(test_tfidf)

negative = 0
positive = 0

for i in range(len(test)):
    print('Text:', test[i])
    if y_pred[i] == 0:
        print('Prediction: Negative review')
        print('Probability: %.2f' % y_prob[i, 0])
        negative += 1
    elif y_pred[i] == 1:
        print('Prediction: Positive review')
        print('Probability: %.2f' % y_prob[i, 1])
        positive += 1

print('negative count : %d positive count : %d' %(negative, positive))

Text: 머스크 "도지코인, 테슬라 결제 시작"…순식간에 20% 폭등
Prediction: Positive review
Probability: 0.62
Text: 테슬라, 전기 픽업트럭 생산 또 연기…“2022년 생산” 홈피 문구 삭제
Prediction: Negative review
Probability: 0.69
Text: 10대 독일 청년 “13개국서 테슬라 차량 25대 원격 해킹”
Prediction: Negative review
Probability: 0.69
Text: 10대 해커에 뚫린 테슬라…원격시동에 차량문 개폐
Prediction: Negative review
Probability: 0.64
Text: 금리인상 우려·빅테크 급락에 美증시 하락…테슬라 6%↓ [출근전 꼭 글로벌브리핑]
Prediction: Negative review
Probability: 0.79
Text: 일론 머스크 "테슬라 인도 시장 진출, 과제 산적"
Prediction: Positive review
Probability: 0.60
Text: 기술주 매도세 다시 살아나…테슬라 7% 가까이 급락(상보)
Prediction: Negative review
Probability: 0.74
Text: 뉴욕증시, 금리인상 우려에 테슬라 등 기술주 하락…나스닥 2.5%↓
Prediction: Negative review
Probability: 0.71
Text: 테슬라 '모델S' 요크 스티어링 휠, 애프터마켓 등장
Prediction: Positive review
Probability: 0.52
Text: 기술주 매도세에 뉴욕증시 하락…테슬라 6.7%↓
Prediction: Negative review
Probability: 0.83
Text: 연준 '매파 변신' 우려에 美증시 급락...테슬라 6.8% MS 4.2%↓
Prediction: Negative review
Probability: 0.64
Text: "금리인상 부담" 나스닥 2.5% 급락...테슬라 6.7%↓[뉴욕마감]

In [14]:
df = df_sample[df_sample['제목'].str.contains('테슬라')]
df.to_csv('news_data.csv')
df

,Unnamed: 0,날짜,언론사,제목,sentiment
0,0,2022.01.14,한국경제,"머스크 ""도지코인, 테슬라 결제 시작""…순식간에 20% 폭등",1.0
1,1,2022.01.14,조선비즈,"테슬라, 전기 픽업트럭 생산 또 연기…“2022년 생산” 홈피 문구 삭제",0.0
2,3,2022.01.14,이데일리,10대 독일 청년 “13개국서 테슬라 차량 25대 원격 해킹”,0.0
3,4,2022.01.14,디지털데일리,10대 해커에 뚫린 테슬라…원격시동에 차량문 개폐,0.0
4,6,2022.01.14,한국경제TV,금리인상 우려·빅테크 급락에 美증시 하락…테슬라 6%↓ [출근전 꼭 글로벌브리핑],0.0
...,...,...,...,...,...
1671,16731,2022.01.15,뉴스1,테슬라 전기트럭 출시 연기에도 1.75% 상승 마감,1.0
1672,16732,2022.01.15,글로벌경제,"테슬라 애플 엔비디아 등 성장주 일제히 상승...뉴욕증시, 보합세 마감",1.0
1673,16733,2022.01.15,테크M,"[글로벌] 전세계 130만명 예약했는데...테슬라, 사이버트럭 '또' 연기",0.0
1674,16736,2022.01.15,초이스경제,"미국증시 자동차주...테슬라 · 루시드 '상승', 리비안 '하락'",1.0


In [33]:
import pandas as pd

# 엑셀 파일 읽기
df = pd.read_csv("news_data.csv", encoding='utf-8')

# sentiment 값이 0인 행 필터링
filtered_data = df[df['sentiment'] == 0]

# 날짜별로 행 개수 세기
zero_count_by_date = filtered_data.groupby('날짜').size()


# sentiment 값이 1인 행 필터링
filtered_data_2 = df[df['sentiment'] == 1]

# 날짜별로 행 개수 세기
one_count_by_date = filtered_data_2.groupby('날짜').size()

for key,value in zero_count_by_date.items():
        if key in one_count_by_date and value < one_count_by_date[key]:
            print(key,'0')
        else:
            print(key,'1')
        



2022.01.01 1
2022.01.02 1
2022.01.03 0
2022.01.04 0
2022.01.05 1
2022.01.06 1
2022.01.07 1
2022.01.08 1
2022.01.09 1
2022.01.10 1
2022.01.11 0
2022.01.12 0
2022.01.13 0
2022.01.14 1
2022.01.15 0
2022.01.16 0
2022.01.17 0
2022.01.18 0
2022.01.19 1
2022.01.20 0
2022.01.21 1
2022.01.22 1
2022.01.23 0
2022.01.24 0
2022.01.25 0
2022.01.26 0
2022.01.27 0
2022.01.28 1
2022.01.29 1
2022.01.30 1
2022.01.31 0
2022.02.01 0
2022.02.02 1
2022.02.03 1
2022.02.04 1
2022.02.06 0
2022.02.07 0
2022.02.08 1
2022.02.09 0
2022.02.10 1
2022.02.11 1
2022.02.12 1
2022.02.13 1
2022.02.14 1
2022.02.15 1
2022.02.16 0
2022.02.17 1
2022.02.18 1
2022.02.19 1
2022.02.20 0
2022.02.21 1
2022.02.22 1
2022.02.23 1
2022.02.24 1
2022.02.25 0
2022.02.26 0
2022.02.28 1
